In [1]:
import numpy as np
from torch.utils.data import DataLoader
from functions import *
from torch.nn import GRU,Conv1d,Linear,ReLU,Sequential,MSELoss,L1Loss,LeakyReLU,Dropout2d,Dropout
from torch.optim import Adam
from pytorch_util import *
from pytorch_models import ConvBatchRelu1D,GRU_NCL

In [2]:
long_page = np.load('../Data/long_page.npy')
long_value = np.load('../Data/long_value.npy')
start_index = np.load('../Data/start_index.npy')
stds = np.load('../Data/stds_log.npy')
means = np.load('../Data/means_log.npy')
long_normal_value = np.load('../Data/long_lognormal_value.npy')

In [3]:
# to account for look back period 365 + 60
start_index = start_index + 430

In [4]:
## to change ##
batch_size = 64
clip = .5

In [5]:
# baseline model
# yhat = long_normal_value[:,803-61:803-60]
# y = long_normal_value[:,803-60:]
# SMAPE_np(y,yhat)
# >> 106.31
# yhat = long_normal_value[:,803-61:803-60]
# y = long_normal_value[:,803-60:]
# np.mean(np.square(y - yhat))
# >>> 1.1114173

In [6]:
# set up data loader
train_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means,useMean=True)
train_gen = DataLoader(train_gen,batch_size,True,num_workers=3,drop_last=True)

val_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means,random=False,useMean=True)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=3)

train_gen_nor = SequenceGenNormalized(long_normal_value,start_index)
train_gen_nor = DataLoader(train_gen_nor,batch_size,True,num_workers=3,drop_last=True)

val_gen_nor = SequenceGenNormalized(long_normal_value,start_index,random=False)
val_gen_nor = DataLoader(val_gen_nor,batch_size)

In [7]:
conv = Sequential(ConvBatchRelu1D(64,128,3),
                  Dropout2d(0.5))

In [8]:
rnn = Sequential(GRU_NCL(3,4,1,batch_first=True,bidirectional=True),
                 Dropout2d(0.2),
                 GRU_NCL(8,8,1,batch_first=True,bidirectional=True),
                 Dropout2d(0.4),
                 GRU_NCL(16,16,1,batch_first=True,bidirectional=True),
                 Dropout2d(0.5),
                 GRU_NCL(32,32,1,batch_first=True,bidirectional=True),
                 Dropout2d(0.5))

In [9]:
seq_model = Sequential(rnn,conv)

In [10]:
linear = Sequential(Linear(128,64),LeakyReLU(0.1,True),Dropout(0.5),Linear(64,1))

In [11]:
convert = Sequential(Linear(118,60),LeakyReLU(0.1,True),Dropout(0.5))

In [12]:
# model trained on normalized data
model_nor = CNN_RNN2seq_normalized_NCL(seq_model,linear,convert).to('cuda:0')

In [13]:
loss_func_nor = loss_func_generator_normalized(MSELoss())
# loss_func_nor = loss_func_generator_normalized(L1Loss())
#loss_func_nor = loss_func_generator_normalized(SMAPE)

In [14]:
opt = Adam(trainable_parameter(model_nor),lr=1e-3)

In [15]:
model_nor = fit(3, model_nor, loss_func_nor, opt, train_gen_nor, val_gen_nor,clip,patience=0)

epoch:0, train_loss:0.8071213854409253, val_loss:0.8893548846244812
epoch:1, train_loss:0.7536742011191314, val_loss:0.8800632357597351
epoch:2, train_loss:0.7430625674329273, val_loss:0.8553664088249207
Training completed in 175.31301593780518s


In [16]:
model_nor = fit(3, model_nor, loss_func_nor, opt, train_gen_nor, val_gen_nor,clip,patience=0,lossBest=0.8553664088249207)

epoch:0, train_loss:0.7327165807003801, val_loss:0.8499274849891663
epoch:1, train_loss:0.7293048627790392, val_loss:0.8558504581451416
epoch:2, train_loss:0.7253457705900459, val_loss:0.8652215600013733
Training completed in 176.03125977516174s


In [17]:
model = CNN_RNN2seq_logNormalized_NCL(seq_model,linear,convert).to('cuda:0')

In [18]:
opt = Adam(trainable_parameter(model),lr=1e-4)

In [19]:
loss_func = loss_func_generator(SMAPE)

In [20]:
model = fit(5, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:36.891070227548866, val_loss:37.8990478515625
epoch:1, train_loss:36.77038871784902, val_loss:37.789756774902344
epoch:2, train_loss:36.57696529250071, val_loss:37.680763244628906
epoch:3, train_loss:36.512990286436725, val_loss:37.64724349975586
epoch:4, train_loss:36.40552719294099, val_loss:37.49324035644531
Training completed in 312.6411306858063s


In [21]:
model = fit(3, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:36.48016745611794, val_loss:37.65636444091797
epoch:1, train_loss:36.4686381078137, val_loss:37.779563903808594
epoch:2, train_loss:36.35943925491886, val_loss:37.52690887451172
Training completed in 187.9349467754364s
